Import tensorflow and reset graph

In [1]:
import tensorflow as tf

# Optimizers

 Gradient descent optimization

$\theta = \theta - \eta \partial J(\theta)/\partial \theta$

Create a gradient descent optimizer by tf

In [2]:
op1 = tf.train.GradientDescentOptimizer(learning_rate=0.01)

Momentum optimization

$m = \beta m - \eta \partial J(\theta) / \partial \theta$

$\theta = \theta + m$

where a typical value of $\beta$ is 0.9

Create a momentum optimizer by tf

In [3]:
op2 = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9)

Nesterov Accelerated Gradient

$m = \beta m - \eta \partial J(\theta + \beta m)/\partial \theta$

$\theta = \theta + m$
where a typical value of $\beta$ is 0.9

Create a Nesterov accelerated gradient optimizer by tf

In [4]:
op3 = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9, use_nesterov=True)

Adam Optimization

This algorithm combine the momentum method and RMSProp (using squared gradient). And it is safe to use the default hyperparameter values by tf without tuning them.

Create a Adam optimizer by tf

In [5]:
op4 = tf.train.AdamOptimizer()

# Learning rate scheduling

Run next cell to build a DNN

In [6]:
#
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")


In the namescope of 'train', define a momentum optimizer that:
1. The initial learning rate is $\eta_0 = 0.1$
2. The learning decays exponentially: $\eta = \eta_0 R^{t/r}$ where $r=10000$ and $R=0.1$

In [7]:
initial_learning_rate = 0.1
decay_step = 10000
decay_rate = 1/10
with tf.name_scope('train'):
    global_step = tf.Variable(0, trainable=False, name='global_step')
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_step, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step)